<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/argoComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install argopy
!pip install cartopy

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from argopy import DataFetcher
import cartopy.crs as ccrs

#User parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

runTime = dt(year=2024, month=10, day=7, hour=12) #1 day before Milton began its most rapid intensification

fHourStep = 3  #Normally 3 for HAFS-A
forecastLength = 48 #Normally 126 for HAFS-A.

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

maxArgoRange = 3

#Retrieve HAFS-A Data

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d_%H"
initStr, initHour = runTime.strftime(runFormat).split("_")

fcastTimes = []
for fHour in range(0, forecastLength+1, fHourStep):
  fcastTimes.append(runTime + pd.Timedelta(hours=fHour))

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

Get TC location from ATCF files

In [ ]:
tcLocs = {}
windRadKey = {"NE":"RAD1", "SE":"RAD2", "SW":"RAD3", "NW":"RAD4"}
windRads = {}

atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"
!wget -O {atcfFile} {atcfURL}
atcf = pd.read_csv(atcfFile, names=cols)

for valid in fcastTimes:

  fHour = int((valid-runTime).total_seconds() / 3600)
  pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
  pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
  tcLocs[valid] = (pointLat, pointLon)

Determine desired argo floats

In [ ]:
argoLoader = DataFetcher()
argoData = argoLoader.region([-95,-80,20,25,0,200.,'2024-10','2024-11']).to_dataframe()

In [ ]:
argos = {}
for valid, tcLoc in tcLocs.items():
    argoDists = ((argoData.LATITUDE-tcLoc[0])**2 + (argoData.LONGITUDE-tcLoc[1])**2)**0.5
    closeArgos = argoData[argoDists < maxArgoRange]
    argoTimeDiffs = np.abs(closeArgos.TIME-valid)
    chosenArgo = closeArgos[argoTimeDiffs==np.min(argoTimeDiffs)]
    argos[valid] = chosenArgo

Get data from HAFS-A output.

In [ ]:
init = runTime
atm = {}
atm[init] = {}

for valid in fcastTimes:

  atm[init][valid] = {}
  initTime = init.strftime("%Y%m%d%H")

  fhour = str(int((valid-init).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour

  atmFile = "hafsa_" + initTime + "_f" + fhour + ".nc"
  atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
  atmData = xr.open_dataset(atmPath)

  point = tcLocs[valid]

  area = {"NE":[point[0], point[1], point[0]+areaRange, point[1]+areaRange], "SE":[point[0]-areaRange, point[1], point[0], point[1]+areaRange],
          "SW":[point[0]-areaRange, point[1]-areaRange, point[0], point[1],], "NW":[point[0], point[1]-areaRange, point[0]+areaRange, point[1]],
          "centered":[point[0]-(0.5*areaRange), point[1]-(0.5*areaRange), point[0]+(0.5*areaRange), point[1]+(0.5*areaRange)], "point":None}[areaAvg]

  for layer in atmData.isobaricInhPa.data:
    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    atm[init][valid][layer] = {}

    if areaAvg == "point": validPoint = atmData.sel(latitude=point[0], longitude=point[1] + 360, isobaricInhPa=layer, method="nearest")
    else: validPoint = atmData.sel(latitude=slice(area[0], area[2]), longitude=slice(area[1]+360, area[3]+360), isobaricInhPa=layer)

    atm[init][valid][layer]["T"] = np.mean(validPoint.t.data)
    if potentialTemp: atm[init][valid][layer]["T"] = np.mean(validPoint.t.data*((1000/layer)**0.28571)) #Formula from Stull R. 2017
    atm[init][valid][layer]["q"] = np.mean(validPoint.q.data) * 1000 #Convert from kg/kg to g/kg
    atm[init][valid][layer]["u"] = np.mean(validPoint.u.data)
    atm[init][valid][layer]["v"] = np.mean(validPoint.v.data)
    atm[init][valid][layer]["gh"] = np.mean(validPoint.gh.data)

In [ ]:
oce = {}
oce[init] = {}

for valid in fcastTimes:
  oce[init][valid] = {}
  initTime = init.strftime("%Y%m%d%H")

  oce[init][valid] = {}

  fhour = str(int((valid-init).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour
  oceFile = "mom6_" + initTime + "_f" + fhour + ".nc"
  ocePath = dataPath + "mom6Output/" + subfolder + oceFile

  if oceFile == 'mom6_2024100800_f000.nc': #This file is missing
    for layer in oceData.z_l.data:
      oce[init][valid][layer] = {"T":np.nan, "s":np.nan, "u":np.nan, "v":np.nan}
    continue

  oceData = xr.open_dataset(ocePath, decode_times=False)

  point = tcLocs[valid]
  if avgType == "wind": areaRange = (2**-0.5) * (windRads[valid]/60) #Convert from n mi to degrees latitude. Does this work for degrees longitude?
  area = {"NE":[point[0], point[1], point[0]+areaRange, point[1]+areaRange], "SE":[point[0]-areaRange, point[1], point[0], point[1]+areaRange],
          "SW":[point[0]-areaRange, point[1]-areaRange, point[0], point[1],], "NW":[point[0], point[1]-areaRange, point[0]+areaRange, point[1]],
          "centered":[point[0]-(0.5*areaRange), point[1]-(0.5*areaRange), point[0]+(0.5*areaRange), point[1]+(0.5*areaRange)], "point":None}[areaAvg]

  for layer in oceData.z_l.data:

    if areaAvg == "point": validPoint = oceData.sel(yh=point[0], yq=point[0], xq=point[1], xh=point[1], z_l=layer, method="nearest")
    else: validPoint = oceData.sel(yq=slice(area[0], area[2]), yh=slice(area[0], area[2]), xq=slice(area[1], area[3]), xh=slice(area[1], area[3]), z_l=layer)

    oce[init][valid][layer] = {}
    oce[init][valid][layer]["T"] = np.mean(validPoint.temp.data) + 273.15 #This is potential temperature. Converted from C to K.
    oce[init][valid][layer]["s"] = np.mean(validPoint.so.data)
    oce[init][valid][layer]["u"] = np.mean(validPoint.uo.data)
    oce[init][valid][layer]["v"] = np.mean(validPoint.vo.data)